<a href="https://colab.research.google.com/github/JulyAnderson/ds-cotas-parlamentares/blob/main/cotas_parlamentares.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisando as Cotas Parlamentares dos Anos de 2019, 2020, 2021, 2022

In [30]:
#importando as ferramentas necessárias
import pandas as pd   # biblioteca para manipulação de dataframes
import chardet as cd  # ferramenta para detecção de codificaçã
from datetime import date #biblioteca para objetos do tipo data
from datetime import datetime #biblioteca para objetos do tipo datetime
import regex as re
import plotly.express as px

In [31]:
#Inicio do processamento
data_e_hora_atuais = datetime.now()
data_e_hora_em_texto= data_e_hora_atuais.strftime('%d%m%Y %H:%M')
print(f'Inicio da leitura dos dados...{data_e_hora_em_texto} \n')

csv= 'despesa_ceaps_2019.csv'

#Busca a codificação do arquivo
with open (csv, 'rb') as arq:
    codificado_com = cd.detect(arq.read())

tipo_codificado = str(codificado_com['encoding']).strip()
print (tipo_codificado)

#Abrindo os arquivos csv e transformando em dataframes
df_22 = pd.read_csv('despesa_ceaps_2022.csv', sep=';',encoding= tipo_codificado,skiprows=1)
df_21 = pd.read_csv('despesa_ceaps_2021.csv', sep=';',encoding= tipo_codificado,skiprows=1)
df_20 = pd.read_csv('despesa_ceaps_2020.csv', sep=';',encoding= tipo_codificado,skiprows=1)
df_19 = pd.read_csv('despesa_ceaps_2019.csv', sep=';',encoding= tipo_codificado,skiprows=1)


# Lista de DataFrames
dataframes = [df_19, df_22, df_21, df_20]

# Verifica se todas as colunas são iguais
eh_equal = all(df.columns.equals(dataframes[0].columns) for df in dataframes)
if eh_equal:
    print('Todas as planilhas possuem as mesmas colunas.')
else:
    colunas_diferentes = [df.columns for df in dataframes if not df.columns.equals(dataframes[0].columns)]
    print(f'Colunas diferentes encontradas: {colunas_diferentes}')


#Realizando a concatenação de todos os dfs em um único arquivo.
df = pd.concat([pd.read_csv(file, sep=';', encoding='latin-1',skiprows=1) for file in ['despesa_ceaps_2019.csv',
                                                            'despesa_ceaps_2020.csv',
                                                            'despesa_ceaps_2021.csv',
                                                            'despesa_ceaps_2022.csv']],ignore_index=True)

#Fim do proccessamento
data_e_hora_atuais   = datetime.now()
data_e_hora_em_texto = data_e_hora_atuais.strftime('%d/%m/%Y %H:%M')
print("Carregamento de dados completo...." + data_e_hora_em_texto)

Inicio da leitura dos dados...02012024 20:33 

Windows-1252
Todas as planilhas possuem as mesmas colunas.
Carregamento de dados completo....02/01/2024 20:33


In [32]:
df.head(3)

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582758,04/01/2019,Despesa com pagamento de energia elétrica do e...,"66,02",2116543
1,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582755,04/01/2019,Despesa com pagamento de energia elétrica do e...,"139,98",2116546
2,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,00119,07/01/2019,Despesa com pagamento de aluguel de imóvel par...,6000,2113817


In [33]:
#obtendo o tamanho do dataframe
print('Dimensões do Data Frame: ', df.shape)

Dimensões do Data Frame:  (69333, 11)


In [34]:
# obtendo informações sobre as colunas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69333 entries, 0 to 69332
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ANO                69333 non-null  int64 
 1   MES                69333 non-null  int64 
 2   SENADOR            69333 non-null  object
 3   TIPO_DESPESA       69333 non-null  object
 4   CNPJ_CPF           69333 non-null  object
 5   FORNECEDOR         69333 non-null  object
 6   DOCUMENTO          66370 non-null  object
 7   DATA               69333 non-null  object
 8   DETALHAMENTO       38583 non-null  object
 9   VALOR_REEMBOLSADO  69333 non-null  object
 10  COD_DOCUMENTO      69333 non-null  int64 
dtypes: int64(3), object(8)
memory usage: 5.8+ MB


### Conversão do Tipo de dado
Considerando as informações das colunas, podemos converter os atributos VALOR_REEMBOLSADO e DATA para tipos de dados mais apropriados.

In [35]:
df['VALOR_REEMBOLSADO']= df.VALOR_REEMBOLSADO.str.replace(',', '.') #troca a vírgula por ponto em toda series
df['VALOR_REEMBOLSADO'] = pd.to_numeric(df.VALOR_REEMBOLSADO, downcast='float') #converte a string para o tipo numérico float64
df['VALOR_REEMBOLSADO'].dtype

dtype('float64')

In [36]:
df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%Y', errors='coerce')
df['DATA'].dtype

dtype('<M8[ns]')

In [74]:
df['VALOR_REEMBOLSADO'].describe()

count     69330.000000
mean       1413.166844
std        3098.235462
min           0.010000
25%         158.140000
50%         426.055000
75%        1570.567500
max      120000.000000
Name: VALOR_REEMBOLSADO, dtype: float64

In [38]:
df[pd.isna(df['DATA'])]

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
10986,2019,1,LÚCIA VÂNIA,"Aluguel de imóveis para escritório político, c...",08.573.731/0001-38,LDE Contabilidade e Administração de Condomíni...,0107007,NaT,Taxa condominial,271.26,2114251
31996,2020,5,PLÍNIO VALÉRIO,Divulgação da atividade parlamentar,27.209.437/0001-96,Excelsior Comunicação Digital Ltda,2020/9,NaT,NaN,1150.00,2144535
37872,2021,11,CARLOS VIANA,"Passagens aéreas, aquáticas e terrestres nacio...",16.978.175/0001-08,ADRIA VIAGENS E TURISMO LTDA,YK1QHD,NaT,"Companhia Aérea: AZUL, Localizador: YK1QHD. Pa...",339.90,2169237


In [39]:
print(f"A média da amostra antes da exclusão dos registros é de {df['VALOR_REEMBOLSADO'].mean()}")
print(f"A média do valor reembolsado dos registros com datas incorretas é: {df[pd.isna(df['DATA'])]['VALOR_REEMBOLSADO'].mean()}")

A média da amostra antes da exclusão dos registros é de 1413.1310983225883
A média do valor reembolsado dos registros com datas incorretas é: 587.0533333333333


Considerando que temos apenas três registros com datas preenchidas de forma incorreta e que a média do VALORE_REEMBOLSADO destes registros é inferior à media da amostra e próxima à mediana, optei por eliminar essas insconsistências.

In [40]:
df.dropna(subset=['DATA'], inplace=True)
print (f"A média da amostra após a exclusão dos registros é de {df['VALOR_REEMBOLSADO'].mean()}")

A média da amostra após a exclusão dos registros é de 1413.1668437905666


In [41]:
df.shape

(69330, 11)

In [42]:
df.head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582758,2019-01-04,Despesa com pagamento de energia elétrica do e...,66.02,2116543
1,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582755,2019-01-04,Despesa com pagamento de energia elétrica do e...,139.98,2116546
2,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,00119,2019-01-07,Despesa com pagamento de aluguel de imóvel par...,6000.00,2113817
3,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.423.963/0001-11,OI MÓVEL S.A.,86161151,2018-12-25,Despesa com pagamento de telefonia para o escr...,316.39,2116541
4,2019,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,007236036,2019-02-04,Despesa com pagamento de energia elétrica para...,99.45,2116550


In [43]:
def is_cpf_or_cnpj(value):
    cleaned_value = re.sub(r'\D', '', value)  # Remover caracteres não numéricos
    if len(cleaned_value) == 11:
        return 'CPF'
    elif len(cleaned_value) == 14:
        return 'CNPJ'
    else:
        return 'Invalido'

# Adicionar a coluna 'PESSOA_FISICA' ao DataFrame
df['PESSOA_FISICA'] = df['CNPJ_CPF'].apply(lambda x: 1 if is_cpf_or_cnpj(x) == 'CPF' else 0)

df['PESSOA_FISICA'].value_counts()

df

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,PESSOA_FISICA
0,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582758,2019-01-04,Despesa com pagamento de energia elétrica do e...,66.02,2116543,0
1,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,006582755,2019-01-04,Despesa com pagamento de energia elétrica do e...,139.98,2116546,0
2,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",004.948.028-63,GILBERTO PISELO DO NASCIMENTO,00119,2019-01-07,Despesa com pagamento de aluguel de imóvel par...,6000.00,2113817,1
3,2019,1,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.423.963/0001-11,OI MÓVEL S.A.,86161151,2018-12-25,Despesa com pagamento de telefonia para o escr...,316.39,2116541,0
4,2019,2,ACIR GURGACZ,"Aluguel de imóveis para escritório político, c...",05.914.650/0001-66,ENERGISA,007236036,2019-02-04,Despesa com pagamento de energia elétrica para...,99.45,2116550,0
...,...,...,...,...,...,...,...,...,...,...,...,...
69328,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,WIXHAI,2022-12-06,"Companhia Aérea: LATAM, Localizador: WIXHAI. P...",2893.04,2191398,0
69329,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,WITOLM,2022-12-09,"Companhia Aérea: GOL, Localizador: WITOLM. Pas...",1180.19,2192272,0
69330,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,THPKVQ,2022-12-20,"Companhia Aérea: TAM, Localizador: THPKVQ. Pas...",2671.90,2192274,0
69331,2022,12,ZEQUINHA MARINHO,"Passagens aéreas, aquáticas e terrestres nacio...",22.052.777/0001-32,Exceller Tour,QNN9HX,2022-12-21,"Companhia Aérea: AZUL, Localizador: QNN9HX. Pa...",1334.31,2192244,0


### Conseguindo algumas informações agrupadas.

In [44]:
print(f'A DataSet possui {len(df["SENADOR"].unique())} diferentes senadores')

A DataSet possui 153 diferentes senadores


In [45]:
df['VALOR_REEMBOLSADO'].describe()

count     69330.000000
mean       1413.166844
std        3098.235462
min           0.010000
25%         158.140000
50%         426.055000
75%        1570.567500
max      120000.000000
Name: VALOR_REEMBOLSADO, dtype: float64

In [46]:
features =  ['SENADOR', 'TIPO_DESPESA', 'PESSOA_FISICA']

for feature in features:
    data = pd.DataFrame(df[feature].value_counts())
    display(data)



,SENADOR
CARLOS VIANA,2135
JAQUES WAGNER,2077
WELLINGTON FAGUNDES,1976
HUMBERTO COSTA,1930
PAULO PAIM,1867
...,...
VIRGINIO DE CARVALHO,3
HÉLIO JOSÉ,3
EDISON LOBÃO,2
VICENTINHO ALVES,1


,TIPO_DESPESA
"Locomoção, hospedagem, alimentação, combustíveis e lubrificantes",25518
"Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.",15205
"Passagens aéreas, aquáticas e terrestres nacionais",14896
"Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos.",5007
Divulgação da atividade parlamentar,4486
"Contratação de consultorias, assessorias, pesquisas, trabalhos técnicos e outros serviços de apoio ao exercício do mandato parlamentar",4105
Serviços de Segurança Privada,113


,PESSOA_FISICA
0,67721
1,1609


In [68]:
#Conseguindo o valor total gastor por senador
df_senador_valor_total = df.groupby('SENADOR')['VALOR_REEMBOLSADO'].sum().reset_index()

# Ordenar o DataFrame pelo valor total reembolsado em ordem decrescente
df_senador_valor_total = df_senador_valor_total.sort_values(by='VALOR_REEMBOLSADO', ascending=False)

# Criar o gráfico de barras com Plotly
fig = px.bar(df_senador_valor_total, x='SENADOR', y='VALOR_REEMBOLSADO',
             title='Valor Total Reembolsado por Senador (Ordenado)',
             labels={'VALOR_REEMBOLSADO': 'Valor Reembolsado (R$)'})

fig.show()

In [63]:
# Encontrando a média de gastos
media = df.groupby('SENADOR')['VALOR_REEMBOLSADO'].sum().mean()
print(f'Média de Gastos: {media}')

# Filtrar apenas os parlamentares que gastaram acima da média
df_senador_valor_total = df_senador_valor_total[df_senador_valor_total['VALOR_REEMBOLSADO'] > media]

# Ordenar o DataFrame pelo valor total reembolsado em ordem decrescente
df_senador_valor_total = df_senador_valor_total.sort_values(by='VALOR_REEMBOLSADO', ascending=False)

# Criar o gráfico de barras com Plotly
fig = px.bar(df_senador_valor_total, x='SENADOR', y='VALOR_REEMBOLSADO',
             title='Valor Total Reembolsado por Senador (Ordenado)',
             labels={'VALOR_REEMBOLSADO': 'Valor Reembolsado (R$)'})

fig.show()

Média de Gastos: 640358.5443137255
